## Part 1: Existing Machine Learning Services

<a href="https://colab.research.google.com/github/peckjon/hosting-ml-as-microservice/blob/master/part1/score_reviews_via_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Obtain labelled reviews

In order to test any of the sentiment analysis APIs, we need a labelled dataset of reviews and their sentiment polarity. We'll use NLTK to download the movie_reviews corpus.

In [3]:
from nltk import download

download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/serj/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

### Load the data

The files in movie_reviews have already been divided into two sets: positive ('pos') and negative ('neg'), so we can load the raw text of the reviews into two lists, one for each polarity.

In [4]:
from nltk.corpus import movie_reviews

# extract words from reviews, pair with label

reviews_pos = []
for fileid in movie_reviews.fileids('pos'):
    review = movie_reviews.raw(fileid)
    reviews_pos.append(review)

reviews_neg = []
for fileid in movie_reviews.fileids('neg'):
    review = movie_reviews.raw(fileid)
    reviews_neg.append(review)

### Connect to the scoring API

Fill in this function with code that connects to one of these APIs, and uses it to score a single review:

* [Amazon Comprehend: Detect Sentiment](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectSentiment.html)
* [Google Natural Language: Analyzing Sentiment](https://cloud.google.com/natural-language/docs/analyzing-sentiment)
* [Azure Cognitive Services: Sentiment Analysis](https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/how-tos/text-analytics-how-to-sentiment-analysis)
* [Algorithmia: Sentiment Analysis](https://algorithmia.com/algorithms/nlp/SentimentAnalysis)

Your function must return either 'pos' or 'neg', so you'll need to make some decisions about how to map the results of the API call to one of these values. For example, Amazon Comprehend can return "NEUTRAL" or "MIXED" for the Sentiment -- if this happens, you may with to inspect the numeric values under the SentimentScore to see whether it leans toward positive or negative.


In [5]:
import boto3
import json

session = boto3.Session(profile_name='serj_tmp')
client = session.client(service_name='comprehend', region_name='us-east-1')


In [7]:
def score_review(review):
    # TBD: call the service and return 'pos' or 'neg'
    response = client.detect_sentiment(
        Text=review[:4000],
        LanguageCode='en')
    
    print(response)
    sentiment = response['Sentiment']
    score = response['SentimentScore']
    posScore = score['Positive']
    negScore = score['Negative']
    
    if (sentiment == 'POSITIVE'):
        return 'pos'
    if (sentiment == 'NEGATIVE'):
        return 'neg'
    if (sentiment == 'NEUTRAL' or sentiment == 'MIXED'):
        if (posScore >= negScore):
            return 'pos'
        return 'neg'
    #return 'pos'

### Score each review

Now, we can use the function you defined to score each of the reviews

In [8]:
results_pos = []
for review in reviews_pos:
    result = score_review(review)
    results_pos.append(result)

results_neg = []
for review in reviews_neg:
    result = score_review(review)
    results_neg.append(result)

{'Sentiment': 'MIXED', 'SentimentScore': {'Positive': 0.26979953050613403, 'Negative': 0.07474786788225174, 'Neutral': 0.07263388484716415, 'Mixed': 0.5828186273574829}, 'ResponseMetadata': {'RequestId': '5511367e-2720-4876-87eb-f9350a83f57b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5511367e-2720-4876-87eb-f9350a83f57b', 'content-type': 'application/x-amz-json-1.1', 'content-length': '159', 'date': 'Mon, 29 Jun 2020 07:32:10 GMT'}, 'RetryAttempts': 0}}
{'Sentiment': 'NEGATIVE', 'SentimentScore': {'Positive': 0.015479560941457748, 'Negative': 0.8817301392555237, 'Neutral': 0.08842789381742477, 'Mixed': 0.014362304471433163}, 'ResponseMetadata': {'RequestId': '6c57d689-5e09-4fc0-9ada-512c672461ab', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '6c57d689-5e09-4fc0-9ada-512c672461ab', 'content-type': 'application/x-amz-json-1.1', 'content-length': '164', 'date': 'Mon, 29 Jun 2020 07:32:10 GMT'}, 'RetryAttempts': 0}}
{'Sentiment': 'NEGATIVE', 'SentimentScore

{'Sentiment': 'POSITIVE', 'SentimentScore': {'Positive': 0.993259608745575, 'Negative': 0.002919308841228485, 'Neutral': 0.003309604013338685, 'Mixed': 0.0005114771192893386}, 'ResponseMetadata': {'RequestId': '640c127a-03e9-44c6-9b08-9639afd760a8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '640c127a-03e9-44c6-9b08-9639afd760a8', 'content-type': 'application/x-amz-json-1.1', 'content-length': '164', 'date': 'Mon, 29 Jun 2020 07:32:14 GMT'}, 'RetryAttempts': 0}}
{'Sentiment': 'POSITIVE', 'SentimentScore': {'Positive': 0.5883514285087585, 'Negative': 0.005045117344707251, 'Neutral': 0.37879395484924316, 'Mixed': 0.02780948206782341}, 'ResponseMetadata': {'RequestId': '20ca5e8a-b0de-4f6b-8162-7c9c36577002', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '20ca5e8a-b0de-4f6b-8162-7c9c36577002', 'content-type': 'application/x-amz-json-1.1', 'content-length': '163', 'date': 'Mon, 29 Jun 2020 07:32:15 GMT'}, 'RetryAttempts': 0}}
{'Sentiment': 'NEUTRAL', 'SentimentS

{'Sentiment': 'POSITIVE', 'SentimentScore': {'Positive': 0.5556450486183167, 'Negative': 0.042811669409275055, 'Neutral': 0.40132057666778564, 'Mixed': 0.00022267318854574114}, 'ResponseMetadata': {'RequestId': '8c032341-afee-434d-8473-04b344c96bf8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '8c032341-afee-434d-8473-04b344c96bf8', 'content-type': 'application/x-amz-json-1.1', 'content-length': '165', 'date': 'Mon, 29 Jun 2020 07:32:18 GMT'}, 'RetryAttempts': 0}}
{'Sentiment': 'NEGATIVE', 'SentimentScore': {'Positive': 0.428545743227005, 'Negative': 0.4625183343887329, 'Neutral': 0.10742741078138351, 'Mixed': 0.0015085384948179126}, 'ResponseMetadata': {'RequestId': 'da0cea31-7927-4bd3-88c7-83cd948a8d99', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'da0cea31-7927-4bd3-88c7-83cd948a8d99', 'content-type': 'application/x-amz-json-1.1', 'content-length': '162', 'date': 'Mon, 29 Jun 2020 07:32:18 GMT'}, 'RetryAttempts': 0}}
{'Sentiment': 'MIXED', 'SentimentSco

{'Sentiment': 'MIXED', 'SentimentScore': {'Positive': 0.32157132029533386, 'Negative': 0.04296443983912468, 'Neutral': 0.2979477047920227, 'Mixed': 0.3375164866447449}, 'ResponseMetadata': {'RequestId': '9b49b974-b097-4138-ba08-e9cc73c35d88', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9b49b974-b097-4138-ba08-e9cc73c35d88', 'content-type': 'application/x-amz-json-1.1', 'content-length': '158', 'date': 'Mon, 29 Jun 2020 07:32:23 GMT'}, 'RetryAttempts': 0}}
{'Sentiment': 'POSITIVE', 'SentimentScore': {'Positive': 0.9984779953956604, 'Negative': 0.0003184269298799336, 'Neutral': 0.001198548823595047, 'Mixed': 5.110122401674744e-06}, 'ResponseMetadata': {'RequestId': '00fb93bc-39b4-48b4-b0fb-3b9a5388f047', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '00fb93bc-39b4-48b4-b0fb-3b9a5388f047', 'content-type': 'application/x-amz-json-1.1', 'content-length': '165', 'date': 'Mon, 29 Jun 2020 07:32:23 GMT'}, 'RetryAttempts': 0}}
{'Sentiment': 'POSITIVE', 'SentimentSco

{'Sentiment': 'POSITIVE', 'SentimentScore': {'Positive': 0.6993663311004639, 'Negative': 0.02506232261657715, 'Neutral': 0.2698339819908142, 'Mixed': 0.005737445317208767}, 'ResponseMetadata': {'RequestId': '6fb4d99f-3794-4578-b7f4-b8b50da36636', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '6fb4d99f-3794-4578-b7f4-b8b50da36636', 'content-type': 'application/x-amz-json-1.1', 'content-length': '162', 'date': 'Mon, 29 Jun 2020 07:32:27 GMT'}, 'RetryAttempts': 0}}
{'Sentiment': 'NEUTRAL', 'SentimentScore': {'Positive': 0.0030701064970344305, 'Negative': 0.00028791732620447874, 'Neutral': 0.9964763522148132, 'Mixed': 0.0001656100357649848}, 'ResponseMetadata': {'RequestId': '483645cc-4d84-4a9f-adcf-50506ece3e25', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '483645cc-4d84-4a9f-adcf-50506ece3e25', 'content-type': 'application/x-amz-json-1.1', 'content-length': '166', 'date': 'Mon, 29 Jun 2020 07:32:27 GMT'}, 'RetryAttempts': 0}}
{'Sentiment': 'POSITIVE', 'Sentime

...

{'Sentiment': 'NEGATIVE', 'SentimentScore': {'Positive': 0.035228580236434937, 'Negative': 0.8831837177276611, 'Neutral': 0.07590623944997787, 'Mixed': 0.005681515671312809}, 'ResponseMetadata': {'RequestId': '3473f5f1-e7d9-4fe3-9df6-7a007c3324cd', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '3473f5f1-e7d9-4fe3-9df6-7a007c3324cd', 'content-type': 'application/x-amz-json-1.1', 'content-length': '164', 'date': 'Mon, 29 Jun 2020 07:39:48 GMT'}, 'RetryAttempts': 0}}
{'Sentiment': 'NEUTRAL', 'SentimentScore': {'Positive': 0.16496451199054718, 'Negative': 0.0632660910487175, 'Neutral': 0.7701222896575928, 'Mixed': 0.0016470649279654026}, 'ResponseMetadata': {'RequestId': '60dd2f36-b6c6-4d55-8ff1-b463f2a18805', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '60dd2f36-b6c6-4d55-8ff1-b463f2a18805', 'content-type': 'application/x-amz-json-1.1', 'content-length': '162', 'date': 'Mon, 29 Jun 2020 07:39:48 GMT'}, 'RetryAttempts': 0}}
{'Sentiment': 'NEUTRAL', 'SentimentSco

{'Sentiment': 'MIXED', 'SentimentScore': {'Positive': 0.19398315250873566, 'Negative': 0.12238212674856186, 'Neutral': 0.11966785788536072, 'Mixed': 0.5639668107032776}, 'ResponseMetadata': {'RequestId': 'd0916396-8f3e-4931-b087-68542d478140', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd0916396-8f3e-4931-b087-68542d478140', 'content-type': 'application/x-amz-json-1.1', 'content-length': '159', 'date': 'Mon, 29 Jun 2020 07:39:52 GMT'}, 'RetryAttempts': 0}}
{'Sentiment': 'NEGATIVE', 'SentimentScore': {'Positive': 0.06648071855306625, 'Negative': 0.7095127701759338, 'Neutral': 0.1940273940563202, 'Mixed': 0.029979130253195763}, 'ResponseMetadata': {'RequestId': '5c84c115-1f37-4550-b32b-9671cfc18b05', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5c84c115-1f37-4550-b32b-9671cfc18b05', 'content-type': 'application/x-amz-json-1.1', 'content-length': '162', 'date': 'Mon, 29 Jun 2020 07:39:52 GMT'}, 'RetryAttempts': 0}}
{'Sentiment': 'NEUTRAL', 'SentimentScore': 

{'Sentiment': 'NEUTRAL', 'SentimentScore': {'Positive': 0.014292733743786812, 'Negative': 0.012778976932168007, 'Neutral': 0.9728860855102539, 'Mixed': 4.223443102091551e-05}, 'ResponseMetadata': {'RequestId': 'b3b43cd3-f557-4375-8824-ccdc7f98ec52', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b3b43cd3-f557-4375-8824-ccdc7f98ec52', 'content-type': 'application/x-amz-json-1.1', 'content-length': '164', 'date': 'Mon, 29 Jun 2020 07:39:56 GMT'}, 'RetryAttempts': 0}}
{'Sentiment': 'NEGATIVE', 'SentimentScore': {'Positive': 0.02818472310900688, 'Negative': 0.9601911306381226, 'Neutral': 0.010311146266758442, 'Mixed': 0.0013129524886608124}, 'ResponseMetadata': {'RequestId': '262fd330-ab0e-4dc0-8f24-a49e144e85dc', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '262fd330-ab0e-4dc0-8f24-a49e144e85dc', 'content-type': 'application/x-amz-json-1.1', 'content-length': '165', 'date': 'Mon, 29 Jun 2020 07:39:57 GMT'}, 'RetryAttempts': 0}}
{'Sentiment': 'NEUTRAL', 'Sentimen

{'Sentiment': 'NEUTRAL', 'SentimentScore': {'Positive': 0.2875852882862091, 'Negative': 0.2942061722278595, 'Neutral': 0.39705318212509155, 'Mixed': 0.021155260503292084}, 'ResponseMetadata': {'RequestId': '0dc842e8-469e-4f73-9832-11430c41afc3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0dc842e8-469e-4f73-9832-11430c41afc3', 'content-type': 'application/x-amz-json-1.1', 'content-length': '161', 'date': 'Mon, 29 Jun 2020 07:40:00 GMT'}, 'RetryAttempts': 0}}
{'Sentiment': 'NEGATIVE', 'SentimentScore': {'Positive': 0.09801720827817917, 'Negative': 0.8590165972709656, 'Neutral': 0.02889670804142952, 'Mixed': 0.0140693886205554}, 'ResponseMetadata': {'RequestId': 'f7bd72b1-3cfb-4b0e-9f38-2c21200248da', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f7bd72b1-3cfb-4b0e-9f38-2c21200248da', 'content-type': 'application/x-amz-json-1.1', 'content-length': '161', 'date': 'Mon, 29 Jun 2020 07:40:00 GMT'}, 'RetryAttempts': 0}}
{'Sentiment': 'POSITIVE', 'SentimentScore'

{'Sentiment': 'NEGATIVE', 'SentimentScore': {'Positive': 0.0018621786730363965, 'Negative': 0.9970971345901489, 'Neutral': 0.0007474826998077333, 'Mixed': 0.0002931357885245234}, 'ResponseMetadata': {'RequestId': 'a0dcfc9e-3a73-4d50-afa6-953b945b4680', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'a0dcfc9e-3a73-4d50-afa6-953b945b4680', 'content-type': 'application/x-amz-json-1.1', 'content-length': '166', 'date': 'Mon, 29 Jun 2020 07:40:05 GMT'}, 'RetryAttempts': 0}}
{'Sentiment': 'NEUTRAL', 'SentimentScore': {'Positive': 0.11615588515996933, 'Negative': 0.05119563266634941, 'Neutral': 0.8325702548027039, 'Mixed': 7.835629367036745e-05}, 'ResponseMetadata': {'RequestId': '31afe393-e279-4617-9a1f-fa62591a14df', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '31afe393-e279-4617-9a1f-fa62591a14df', 'content-type': 'application/x-amz-json-1.1', 'content-length': '162', 'date': 'Mon, 29 Jun 2020 07:40:05 GMT'}, 'RetryAttempts': 0}}
{'Sentiment': 'POSITIVE', 'Sentim

### Calculate accuracy

For each of our known positive reviews, we can count the number which our function scored as 'pos', and use this to calculate the % accuracy. We repeaty this for negative reviews, and also for overall accuracy.

In [11]:
correct_pos = results_pos.count('pos')
accuracy_pos = float(correct_pos) / len(results_pos)
correct_neg = results_neg.count('neg')
accuracy_neg = float(correct_neg) / len(results_neg)
correct_all = correct_pos + correct_neg
accuracy_all = float(correct_all) / (len(results_pos)+len(results_neg))

print('Positive reviews: {}% correct'.format(accuracy_pos*100))
print('Negative reviews: {}% correct'.format(accuracy_neg*100))
print('Overall accuracy: {}% correct'.format(accuracy_all*100))

Positive reviews: 75.3% correct
Negative reviews: 56.99999999999999% correct
Overall accuracy: 66.14999999999999% correct
